In [1]:
# Minkowski distance:
from math import *
import numpy as np


def p_root(value, root):
    root = 1 / float(root)
    return round ((value) ** (root), 3)

def minkowski_distance(x, y, p):
    return (p_root(sum(pow(abs(a-b), p) for a, b in zip(x, y)), p))

p = 1

In [2]:
# Load data and define K:

import pandas as pd
from scipy.stats import zscore
from collections import Counter

df = pd.read_csv("datasets/1.data")
df = df.fillna(0)
number_of_columns = len(df.columns)
list_of_classes = df['class'].to_list()

# Number of unique classes:
classes = Counter(list_of_classes).keys()
K = len(classes)
print('K', K)

optimal_labels = []
for item in list_of_classes:
    optimal_labels.append(list(classes).index(item))

print('optimal_labels', optimal_labels)

# Remove class from dataframe
data = df.drop("class", axis=1)

K 3
optimal_labels [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [3]:
n_samples, n_features = data.shape

def matrix_of_distances(p = 1):
    distances = []
    for index, row in data.iterrows():
        sample = row.values.tolist()

        distance = [
            minkowski_distance(sample, row.values.tolist(), p) for index, row in data.iterrows()
        ]

        distances.append(distance)

    return distances

matrix = matrix_of_distances(p)

In [4]:
# Use sklearn do evaluate our solution
from sklearn.cluster import KMeans
from operator import itemgetter
import time
from sklearn.metrics import silhouette_score, adjusted_rand_score



def calc_sklearn_radius():
    candidates = []
    for center_idx, center in enumerate(kmeans.cluster_centers_):
        options = []
        for idx, cluster in enumerate(kmeans.labels_):
            if center_idx == cluster:
                options.append(minkowski_distance(center, data.iloc[idx].values.tolist(), p))
        candidates.append(max(options))
        
    return max(candidates)


start_time = time.time()

kmeans = KMeans(n_clusters=K, random_state=0, max_iter=300).fit(data)
sklearn_radius = calc_sklearn_radius()
sklearn_labels = kmeans.labels_
sklearn_silhouette = round(silhouette_score(data, kmeans.labels_), 5)
rand_score = round(adjusted_rand_score(optimal_labels, sklearn_labels), 5)

sklearn_end_time = round(time.time() - start_time, 5)

print('Optimal radius', sklearn_radius)
print('Optimal labels', sklearn_labels)
print('Optimal silhouette', sklearn_silhouette)
print('Optimal rand index', rand_score)
print('Optimal time', sklearn_end_time)


Optimal radius 2.977
Optimal labels [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 2 2 2 0 2 2 2 2
 2 2 0 0 2 2 2 2 0 2 0 2 0 2 2 0 0 2 2 2 2 2 0 2 2 2 2 0 2 2 2 0 2 2 2 0 2
 2 0]
Optimal silhouette 0.55259
Optimal rand index 0.73024
Optimal time 0.04837


In [6]:
def K_Means(distances):

    # If we have more centers than elements, return the dataset itself
    if K > n_samples:
        return data
    
    # Starting with a random center 
    centroids = np.random.choice(n_samples, 1).tolist()
    # print('First center:', centroids[0])
    
    # Select s that maximize dist(s, C)
    while len(centroids) < K:
        candidates = []
        for idx_row in range(n_samples):
            s_is_centroid = [idx_row == centroid for centroid in centroids]
            if not any(s_is_centroid):
                most_distant_centroid = max(distances[idx_row][centroid] for centroid in centroids)
                candidates.append({'distance': most_distant_centroid, 'index': idx_row})

        if len(candidates) > 0:
            max_item = max(candidates, key=itemgetter('distance'))
            s = max_item['index']
            centroids.append(s)
        
    return centroids

# Calcule clusters
def calc_clusters(centers, distances):
    # Initialize clusters with empty array
    clusters = [[] for _ in range(K)]
    labels = []
    for idx_row in range(n_samples):
        candidates = []
        for idx, center in enumerate(centers):
            candidates.append({'distance': distances[idx_row][center], 'center_index': idx})
        closest_centroid = min(candidates, key=itemgetter('distance'))
        clusters[closest_centroid['center_index']].append(idx_row)
        labels.append(closest_centroid['center_index'])

    return {'clusters': clusters, 'labels': labels}


# Calculate radius
def calc_radius(clusters, distances, C):
    candidates = []
    for center_index, cluster in enumerate(clusters):
        candidates.append(max(distances[C[center_index]][element] for element in cluster))

    return max(candidates)
 
def kmeans_calc():
    C = K_Means(matrix)
    print('Centers', C)

    result = calc_clusters(C, matrix)
    radius = calc_radius(result['clusters'], matrix, C)

    return {'radius': radius, 'labels': result['labels']}
    
    
def execute_calculations(max_iterations = 1): 
    for _ in range(max_iterations):
        start_time = time.time()
        result = kmeans_calc()
        end_time = round(time.time() - start_time, 5)
        silhouette = round(silhouette_score(data, result['labels']), 5)
        rand_score = round(adjusted_rand_score(optimal_labels, result['labels']), 5)
        
        print('Radius', result['radius'])
        print('Labels', result['labels'])
        print('Silhouette', silhouette)
        print('Adjusted Rand', rand_score)
        print('Time', end_time)
        print('----------------')
    
    
execute_calculations(30)

Centers [77, 22, 118]
Radius 4.9
Labels [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Silhouette 0.53681
Adjusted Rand 0.53791
Time 0.00119
----------------
Centers [142, 22, 118]
Radius 4.0
Labels [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 